# Veeper: Create, Run, and Collate

This is a generalized code that can bring in any spectra, save it to its own components folder, and save the corresponding fits

Install: https://github.com/mattcwilde/veeper


Here are some notes for the future

#### Running this code in Step 3 will produce a FitInspection.pdf file with a preview of your voigt fits: 
- Red line = total 
- dashed line = particular component 
- the order is the redshift 

#### If the file needs to be changed (editing.ipynb): 

In [1]:
##--- Step one 
from joebvp.utils import pyigm_to_veeper
from joebvp.VPmeasure import batch_fit
from joebvp import VPmeasure
import glob
import os
import astropy.units as u

read_sets: Using set file -- 
  /Users/samgarza/linetools/linetools/lists/sets/llist_v1.3.ascii
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/morton03_table2.fits.gz
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/verner96_tab1.fits.gz
joebvp.makevoigt: No local joebvp_cfg.py found, using default cfg.py file from joebvp.
joebvp.VPmeasure: No local joebvp_cfg.py found, using default cfg.py file from joebvp.
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/morton03_table2.fits.gz
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/verner96_tab1.fits.gz
joebvp.makevoigt: No local joebvp_cfg.py found, usin

#### Paths for each of my spectra: 
- ##### NGC 3489 / J1101: 
    - json_path =  '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1101+1429/SamIDs_Nov28_2022.json' 
    - norm = 'J1101+1429_nbin3_jwnorm.fits'

- ##### NGC 4026 / J1159: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1159+5106/SamIDs_meth3_Nov28_2022.json'
    - norm = 'J1159+5106_nbin3_jwnorm_meth3.fits'
    
- ##### NGC 4736 / J1249: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1249+4122/SamIDs_Nov28_2022.json'
    - norm = 'J1249+4122_nbin3_visit12_jwnorm.fits'
    
- ##### NGC 3414 / 1051: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1051+2805/SamIDs_Nov29_2022.json'
    - norm = 'J1051+2805_nbin3_jwnorm.fits'
    
- ##### NGC 3627 / J1123: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123+1257/SamIDs_Nov29_2022.json'
    - norm = 'J1123+1257_nbin3_BASIC_jwnorm.fits'

- ##### NGC 1097 / J0246: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J0246-3007/SamIDs_Nov28_2022.json'
    - norm = 'J0246-3007_nbin3_allvisits_jwnorm.fits'

- ##### NGC 4564 / J1235: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1235+1123/SamIDs_Nov28_2022.json'
    - norm = 'J1235+1123_twovisits_m3_jwnorm.fits'

In [5]:
#Step 2: Imput the specifications 
    # basically takes the gueses and the norm to start the process
    #dv_expand is how much you can expand the redlines to be of some problem 

#These will change for each galaxy/line of sight  
#path to specific .json file
json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123+1257/SamIDs_Nov29_2022.json'

# normalization file that you want to use
norm = 'J1123+1257_nbin3_BASIC_jwnorm.fits'
norm_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123+1257/J1123+1257_nbin3_BASIC_jwnorm.fits' #path to normalization file 

pyigm_to_veeper(json_path, norm, dv_expand=50*u.km/u.s)

Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


In [6]:
input_files = glob.glob('component_groups/*.txt')

#This was 10 in the other one but I want to make this 20 since it skipped something that was 15 before 
max_len = 20 #skip over files where there are more than this many lines to fit 
redo = False

to_do = []
for input_file in input_files:
    if 'rchi2' in input_file:
        continue
    if not redo:
        #already done?
        if os.path.exists(input_file.replace('.txt', '.VP')): 
            print('Skipping: Already done {}'.format(input_file))
            continue
    
    #too long?
    with open(input_file, 'r') as f:
        length = len(f.readlines())
        if length > max_len:
            print('Skipping: Too long ({} rows) {}'.format(length, input_file))
            continue
            
    to_do.append(input_file)

In [7]:
###--- Step 3: Run the fit
VPmeasure.batch_fit(norm_path, filelist=to_do, filepath='./component_groups/')

`ftol` termination condition is satisfied.
Function evaluations 18, initial cost 9.6514e+01, final cost 7.0484e+01, first-order optimality 1.88e-03.

Fit results: 

1393.76	 -0.000410	 13.484	 72.600	 67.175
 	  	  	 0.344	 42.884	 46.878 

1402.77	 -0.000410	 13.484	 72.600	 67.175
 	  	  	 0.0	 0.0	 0.0 

1393.76	 0.000190	 13.156	 55.600	 -9.3405
 	  	  	 0.717	 44.128	 54.119 

1402.77	 0.000190	 13.156	 55.600	 -9.3405
 	  	  	 0.0	 0.0	 0.0 


Reduced chi-squared: 1.136832
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_0.txt
Line parameters written to:
component_groups/input_group_0.VP
Wrote spectrum to component_groups/input_group_0_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_0_inspect.fits


/Users/samgarza/linetools/linetools/spectra/lsf.py:551: UserWarning: LSF may result from extrapolation outside wavelength range characterized for current grating.
  "LSF may result from extrapolation outside wavelength range characterized for current grating.")


`ftol` termination condition is satisfied.
Function evaluations 6, initial cost 6.9352e+01, final cost 6.6737e+01, first-order optimality 1.18e-03.

Fit results: 

1190.42	 0.200120	 13.340	 53.446	 8.2192
 	  	  	 0.052	 8.094	 5.661 

1193.29	 0.200120	 13.340	 53.446	 8.2192
 	  	  	 0.0	 0.0	 0.0 

1260.42	 0.200120	 13.340	 53.446	 8.2192
 	  	  	 0.0	 0.0	 0.0 

1304.37	 0.200120	 13.340	 53.446	 8.2192
 	  	  	 0.0	 0.0	 0.0 


Reduced chi-squared: 0.767092
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_1.txt
Line parameters written to:
component_groups/input_group_1.VP
Wrote spectrum to component_groups/input_group_1_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_1_inspect.fits
`ftol` termination condition is satisfied.
Function evaluations 15, initial cost 3.6219e+01, final cost 3.5192e+01, first-order optimality 4.68e-04.

Fit results: 

1526.71	 0.002480	 13.472	 26.700	 -24.959
 	  	  	 0.251	 24.449	 15.925 


Reduced chi-squared: 2.427048
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_3.txt
Line parameters written to:
component_groups/input_group_3.VP
Wrote spectrum to component_groups/input_group_3_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


...

Wrote spectrum to component_groups/input_group_6_inspect.fits
There are no valid pixels to fit in the region. Moving on.
`ftol` termination condition is satisfied.
Function evaluations 507, initial cost 1.2946e+02, final cost 8.5136e+01, first-order optimality 2.91e-03.

Fit results: 

1548.2	 -0.000160	 13.496	 4.0000	 1.6322
 	  	  	 0.579	 4.878	 5.717 

1550.78	 -0.000160	 13.496	 4.0000	 1.6322
 	  	  	 0.0	 0.0	 0.0 

1548.2	 0.000250	 14.126	 49.492	 -52.041
 	  	  	 0.063	 9.986	 6.687 

1550.78	 0.000250	 14.126	 49.492	 -52.041
 	  	  	 0.0	 0.0	 0.0 

1548.2	 0.002550	 13.650	 5.3002	 -4.2306
 	  	  	 0.61	 5.278	 3.866 

1550.78	 0.002550	 13.650	 5.3002	 -4.2306
 	  	  	 0.0	 0.0	 0.0 

1548.2	 0.002860	 13.791	 52.300	 -27.086
 	  	  	 0.132	 23.021	 13.681 

1550.78	 0.002860	 13.791	 52.300	 -27.086
 	  	  	 0.0	 0.0	 0.0 

1548.2	 0.003150	 13.040	 4.0000	 -17.35
 	  	  	 0.573	 13.301	 10.917 

1550.78	 0.003150	 13.040	 4.0000	 -17.35
 	  	  	 0.0	 0.0	 0.0 


Reduce

Wrote spectrum to component_groups/input_group_4_inspect.fits

VPmeasure: 6/7 fits converged, 0/7 failed (see log for details).

VPmeasure: concatenating individual outputs and creating figures for inspection.
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


/Users/samgarza/linetools/linetools/spectra/lsf.py:551: UserWarning: LSF may result from extrapolation outside wavelength range characterized for current grating.
  "LSF may result from extrapolation outside wavelength range characterized for current grating.")
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr

Wrote spectrum to FitInspection.fits
VPmeasure: Done.


/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(),

In [8]:
#make the directory to hold creates files for each qso/LOS 

parent_path = '/Users/samgarza/Dropbox/Mac/Documents/COS-Holes/veeper_and_voigt/' #where I want the created files to be put in general 
qso_name = 'J1123/' #need to change this to match each QSO LOS name; where I want the created files to be put in specifically 

#set directory name
directory = qso_name
# Path
path = os.path.join(parent_path, directory)
#Make it happen
os.mkdir(path)

In [ ]:
#Step 4: Collate
import pandas as pd

vp_file_list = glob.glob('component_groups/*.VP')
vp_file_list

In [ ]:
vp_dfs = []

for vp_file in vp_file_list:
    vp_df = pd.read_csv(vp_file, sep='|')
    vp_dfs.append(vp_df)
    
all_vp_df = pd.concat(vp_dfs, ignore_index=True)
all_vp_df

In [ ]:
#Save collate informtion to csv file 
all_vp_df.to_csv(path + 'all_vp.csv')

In [ ]:
import shutil

#Move over the component_groups
shutil.move(parent_path + 'component_groups/', path)

In [ ]:
#list of created files that need to be moved over to QSO LOS specific directory 
files_to_move = ['_VP_log.dat', 'all_VP.txt', 'compiledVPoutputs.dat', 'FitInspection.fits', 'FitInspection.pdf']

# iterate files
for file in files_to_move:
    # construct full file path
    source = parent_path + file
    destination = path + file
    # move file
    shutil.move(source, destination)
    print('Moved:', file)